<a href="https://colab.research.google.com/github/435wldms/essa/blob/main/20230929_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 비지도 학습

* 군집: 클러스터로 비슷한 샘플끼리 모으기
* 이상치 탐지: '정상' 데이터가 어떻게 보이는지를 학습
* 밀도 추정: 데이터셋 생성 확률 과정의 확률 밀도 함수를 추정

**9.1 군집**

* 비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룸으로 할당하는 작업
* 분류는 레이블이 존재, 군집은 레이블이 존재하지 않을 때 수행
* 고객 분류, 데이터 분석, 차원 축소 기법, 이상치 탐지, 준지도 학습, 검색 엔진, 이미지 분할

**9.1.1 k-평균**

* 각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플을 할당

In [1]:
from sklearn.datasets import make_blobs
import numpy as np


blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [2]:
from sklearn.cluster import KMeans
k=5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


-> 각 샘플은 다섯 개의 클러스터 중 하나에 할당되고, 각 샘플의 레이블이 알고리즘이 샘플에 할당한 클러스터의 인덱스


In [3]:
# 훈련된 샘플의 레이블
y_pred

array([2, 3, 1, ..., 0, 1, 3], dtype=int32)

In [4]:
# 알고리즘이 찾은 센트로이드 확인
kmeans.cluster_centers_

array([[-2.79290307,  2.79641063],
       [ 0.20876306,  2.25551336],
       [-2.80037642,  1.30082566],
       [-2.80389616,  1.80117999],
       [-1.46679593,  2.28585348]])

In [5]:
X_new = np.array([[0,2],[3,2],[-3,3],[-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 0, 0], dtype=int32)

* 샘플은 대부분 적절한 클러스터에 잘 할당되었으나 경계 부근에 있는 일부는 잘못 부여됨
* 알고리즘은 클러스터의 크기가 많이 다르면 잘 작동하지 않음
* 소프트 군집: 클러스터마다 샘플에 점수를 부여하는 것 -> 그 중 하나가 샘플과 센트로이드 사이의 거리

In [6]:
kmeans.transform(X_new)

array([[2.9042344 , 0.32995317, 2.88633901, 2.81093633, 1.49439034],
       [5.84739223, 2.80290755, 5.84236351, 5.80730058, 4.4759332 ],
       [0.29040966, 3.29399768, 1.71086031, 1.21475352, 1.69136631],
       [0.36159148, 3.21806371, 1.21567622, 0.72581411, 1.54808703]])

* **k-평균 알고리즘**: 센트로이드를 랜덤하게 초기화하고 샘플에 레이블을 할당하고 센트로이드를 업데이트하고 반복 ...
* 랜덤한 센트로이드 초기화에 따라 솔루션의 결과가 달라짐

* 센트로이드 초기화 방법: 센트로이드 위치를 근사하게 알 수 있다면 리스트에 넘파이 배열을 지정하고 n_init을 1로 설정

In [7]:
from sklearn.cluster import KMeans
import numpy as np
good_init = np.array([[-3,3],[-3,2],[-3,1],[-1,2],[0,2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

* 랜덤 초기화를 다르게 하여 여러 번 알고리즘을 수행하고 가장 좋은 솔루션을 선택하는 방법이 있음 -> n_init으로 개수 조절, 성능 지표를 통해 최선의 솔루션 찾기
* kmeans 클래스는 알고리즘을 n_init번 실행하여 이너셔(평균 제곱 거리)가 가장 낮은 모델 반환
* 값이 궁금하면 inertia_ 인스턴스 변수로 이너셔 확인할 수 있음

In [ ]:
kmeans.inertia_

In [ ]:
# 이너셔의 음숫값 반환 (score 메서드가 더 높은 값이 좋음)
kmeans.score(X)

* k-평균 알고리즘을 향상시킨 k평균++ 알고리즘 발전
* 다른 센트로이드와 거리가 먼 센트로이드를 선택하는 것

**k-평균 속도 개선과 미니배치 k-평균**

* 불필요한 거리 계산을 많이 피함으로서 알고리즘의 속도를 상당히 높일 수 있음 -> 삼각부등식 사용하고 샘플과 센트로이드 사이의 거리를 위한 상한&하한선 유지

* 전체 데이터셋을 이용해 반복하지 않고 미니배치를 사용해 센트로이드 이동

In [8]:
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=5)

* 데이터셋이 메모리에 들어가지 않으면 memmap 클래스를 사용하거나, partial_fit() 메서드에 한번에 전달

* 미니배치 k-평균 알고리즘이 일반 k-평균 알고리즘보다 훨씬 빠르지만 이너셔는 일반적으로 조금 더 나쁨

**최적의 클러스터 개수 찾기**

* 이너셔가 작은 모델이 최고는 아님(클러스터가 늘어날수록 가까운 센트로이드에 더 가까워지기 때문)

* 실루엣 점수: 최선의 클러스터 개수를 선택하는 방법
* +1에 가까울수록 클러스터링을 잘 한 것

In [ ]:
# 실루엣 계수 확인
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

* 실루엣 다이어그램을 통해 클러스터와 계숫값으로 정렬한 그림그리기

**9.1.2 k-평균의 한계**

* 최적이 아닌 솔루션을 피하려면 알고리즘을 여러 번 실행해야 하고, 클러스터 개수를 지정해야 하며, 클러스터 간의 크기나 밀집도가 다르거나 원형이 아니면 잘 작동하지 않음
* k-means 수행 전 스케일링을 실시하는 게 좋음

**9.1.3 군집을 사용한 이미지 분할**

* 이미지 분할: 이미지를 세그먼트 여러 개로 분할
* 색상 분할을 이용해 동일한 색상을 가진 픽셀을 같은 세그먼트에 할당

In [9]:
import os
import urllib.request

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

('./images/unsupervised_learning/ladybug.png',
 <http.client.HTTPMessage at 0x7dc839179180>)

In [10]:
from matplotlib.image import imread
image = imread(os.path.join('images','unsupervised_learning','ladybug.png'))
image.shape

(533, 800, 3)

* RGB 색상의 긴 리스트로 변환한 다음 K-평균을 사용해 이 색상을 클러스터로 모음

In [11]:
X = image.reshape(-1,3)
kmeans = KMeans(n_clusters=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


* 8개보다 클러스터 개수를 작게 하면 무당벌레의 화려한 빨간 색이 독자적인 클래스를 만들지 못해 주위 색에 합쳐짐

**9.1.4 군집을 사용한 전처리**

* 8x8 크기 흑백 이미지 1797개 불러옴

In [12]:
from sklearn.datasets import load_digits
X_digits,y_digits = load_digits(return_X_y=True)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [14]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
log_reg.score(X_test, y_test)

0.9533333333333334

* k-means를 전처리 단계로 사용해 더 좋아지는지 확인
* 훈련 세트를 50개의 클러스터로 모으고, 이미지를 50개 클러스터까지의 거리로 바꾼 후 회귀 모델 적용

In [16]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('kmeans', KMeans(n_clusters=50)),
    ('log_reg', LogisticRegression()),
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [17]:
pipeline.score(X_train, y_train)

0.9888641425389755

* 임의로 k값을 정하지 않고 더 좋은 k값을 찾기 위해 GridSearchCV 사용

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.3s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   0.9s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   0.8s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [19]:
grid_clf.best_params_
grid_clf.score(X_test, y_test)

0.96

**9.1.5 군집을 사용한 준지도 학습**

* 레이블이 없는 데이터가 많고 레이블이 있는 데이터는 적을 때 사용

In [20]:
# 50개 샘플만 사용
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled],y_train[:n_labeled])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
log_reg.score(X_test, y_test)

0.8022222222222222

-> 적은 데이터셋을 사용했기 때문에 정확도가 더 낮음
>
-> 개선하기 위해, 50개의 클러스터로 훈련 세트를 모은 후 각 클러스터 별로 센트로이드에 가장 가까운 이미지 찾기

In [22]:
k=50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


* 사진을 보고 직접 할당

In [23]:
y_train[representative_digit_idx]

array([3, 1, 4, 5, 0, 2, 1, 7, 8, 6, 6, 1, 8, 2, 4, 7, 5, 2, 5, 1, 9, 6,
       7, 8, 1, 0, 9, 4, 3, 8, 9, 5, 9, 9, 5, 2, 2, 4, 7, 0, 6, 3, 7, 4,
       1, 6, 2, 8, 5, 3])

In [24]:
y_representative_digits = np.array([3, 1, 4, 5, 0, 2, 1, 7, 8, 6, 6, 1, 8, 2, 4, 7, 5, 2, 5, 1, 9, 6,
       7, 8, 1, 0, 9, 4, 3, 8, 9, 5, 9, 9, 5, 2, 2, 4, 7, 0, 6, 3, 7, 4,
       1, 6, 2, 8, 5, 3])

In [25]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9288888888888889

-> 대표 샘플에 레이블을 할당함으로써 수치가 많이 개선됨

* 여기서 더 나아가 동일한 클러스터에 있는 모든 샘플로 전파 : 레이블 전파

In [26]:
y_train_propagated = np.empty(len(X_train),dtype=np.int32)
for i in range(k):
  y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [27]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test,y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9577777777777777

-> 크게 성능이 향상되지 않음 because 대표 샘플의 레이블을 클러스터의 모든 샘플에 전파했기 때문
>
-> 20%에만 전파

In [30]:
percentile_closest = 20
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
  in_cluster = (kmeans.labels_ == i)
  cluster_dist = X_cluster_dist[in_cluster]
  cutoff_distance = np.percentile(cluster_dist, percentile_closest)
  above_cutoff = (X_cluster_dist > cutoff_distance)
  X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [31]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test,y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9288888888888889

-> 레이블된 샘플 50개만으로 얻은 결과치고 높은ㅅ ㅜ치

In [32]:
np.mean(y_train_partially_propagated==y_train[partially_propagated])

1.0

**9.1.6 DBSCAN**

* 작동방식: 알고리즘이 각 샘플에서 작은 거리인 e 내에 몇 개의 샘플이 놓여 있는지 세기(e이웃) -> e이웃 내에 적어도 min_samples개 샘플이 있다면 이를 핵심 샘플로 간주 -> 핵심 샘플의 이웃에 있는 샘플은 동일한 클러스터에 속함, 이웃에는 다른 핵심 샘플이 포함될 수 있음 -> 핵심 샘플도, 이웃도 아니면 이상치
* 이 알고리즘은 모든 클러스터가 충분히 밀집되어 있고, 밀집되지 않은 지역과 잘 구분될 때 좋은 성능을 냄

In [1]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [35]:
dbscan.labels_

array([ 0,  1,  4, 10,  5,  2,  3,  1,  4,  0,  5, -1,  0,  0,  0,  1,  0,
        6,  2,  3,  0,  2,  0,  4,  5,  0,  0,  0,  1,  1,  0,  5, -1,  2,
        7,  0,  0,  0,  1,  2,  2,  7,  1,  5,  1,  4,  2,  7,  1,  0,  7,
        0,  0, -1,  1,  2,  0,  4,  3, -1,  5,  3,  7,  4,  7, -1,  3,  0,
       -1,  6,  3,  5,  0,  5,  0, -1,  2,  5,  5,  5,  3,  5,  0,  6,  0,
        0,  1,  5,  0,  5,  6,  2, -1,  3,  0,  3,  2,  6,  0,  4,  1,  3,
        7,  3,  2,  4,  4,  4,  0,  3,  5,  0,  6,  6,  1,  0,  2,  1,  0,
        0, -1,  3,  0,  8,  3,  4,  0,  4,  2,  0,  0,  5,  3,  0,  6,  2,
        1,  3,  0,  4,  1,  1,  1,  5, -1,  4,  4,  3,  8,  5,  2,  6,  0,
        5,  1,  3, -1,  9, -1, -1,  0,  3,  5,  4,  2,  2,  6,  0,  9,  3,
        6,  1,  0,  0,  4,  4,  1,  5,  5,  5,  5,  3,  1,  2,  4,  5,  3,
        3,  3,  5,  2,  0,  0,  2,  3,  4,  0,  4,  3,  4,  5,  1, -1,  2,
        0,  7,  2,  5,  6,  7,  2,  1,  6,  6,  6,  0,  2,  9,  3,  4,  4,
        3,  0,  0,  0,  4

-> 인덱스가 -1인 것은 이상치를 의미

In [36]:
# 핵심 샘플의 인덱스: core_sample_indices_
len(dbscan.core_sample_indices_)

812

In [37]:
dbscan.core_sample_indices_

array([  0,   1,   5,   6,   7,   8,   9,  10,  12,  13,  14,  15,  16,
        17,  18,  19,  21,  22,  23,  24,  25,  26,  28,  29,  30,  31,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  48,  49,  50,  51,  54,  55,  56,  57,  58,  60,  61,  62,
        63,  64,  67,  69,  70,  71,  72,  73,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  86,  87,  88,  89,  90,  91,  93,  94,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 112, 113, 114, 115, 117, 118, 121, 122, 123, 124, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 136, 137, 139, 140, 141, 143,
       145, 146, 147, 150, 151, 152, 153, 154, 155, 157, 160, 161, 162,
       163, 164, 165, 166, 167, 169, 170, 171, 173, 174, 175, 176, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192,
       193, 194, 195, 196, 197, 198, 199, 200, 201, 203, 204, 206, 207,
       208, 209, 210, 213, 214, 215, 216, 217, 218, 219, 220, 22

In [38]:
dbscan.components_

array([[ 1.4707732 , -0.31787865],
       [ 0.73456822, -0.53580705],
       [ 0.88766022,  0.49882393],
       ...,
       [ 0.78219059, -0.46629286],
       [ 1.94019259,  0.31679287],
       [ 0.1648903 ,  0.93551935]])

* dbscan은 predict가 없고 fit_predict만 존재 -> 새로운 샘플에 대해 클러스터를 예측할 수 없음
* 알아서 필요한 예측기를 선택해야 함

In [40]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [41]:
X_new = np.array([[-0.5,0],[0,0.5],[1,-0.1],[2,1]])
knn.predict(X_new)
knn.predict_proba(X_new)

array([[0.  , 0.  , 0.  , 0.16, 0.  , 0.38, 0.  , 0.  , 0.  , 0.44, 0.02],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.24, 0.76, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

* 훈련 세트에 이상치가 없어서 분류기는 항상 클러스터 하나를 선택해서 반환 -> 최대 거리를 사용하면 멀리 떨어진 샘플을 이상치로 간단히 분류할 수 있음(kneighbors())

In [42]:
# kneighbors(): 가장 가까운 이웃 k개의 거리와 인덱스 반환
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist > 0.2] = -1
y_pred.ravel()

array([-1,  3,  2, -1])

-> dbscan은 클러스터의 모양이나 개수에 상관없이 감지할 수 있으나 클러스터 간의 밀집도가 크게 다르면 모든 클러스터를 올바르게 잡아내기 어려움

**9.1.7 다른 군집 알고리즘**

* 병합 군집: 클러스터 계층을 밑바닥부터 위로 쌓아 구성함, 반복마다 병합 군집은 인접한 클러스터 쌍을 연결하는데 병합된 클러스터 쌍을 트리로 그리면 클러스터의 이진 트리를 얻을 수 있음
* BIRCH: 대규모 데이터셋을 위해 고안됨, 훈련 과정에서 새로운 샘플을 클러스터에 빠르게 할당할 수 있는 정보를 담은 트리 구조를 형성
* 평균-이동: 각 샘플을 중심으로 하는 원을 그리고 원마다 안에 포함된 모든 샘플의 평균을 구한 후에 원의 중심을 평균점으로 이동시킴, 모든 원이 움직이지 않을 때까지 계속 평균이동을 반복
* 유사도 전파: 샘플이 자신을 대표할 수 있는 비슷한 샘플에 투표함, 알고리즘이 수렴하면 각 대표와 투표한 샘플이 클러스터를 형성
* 스펙트럼 군집: 샘플 사이의 유사도 행렬을 받아 저차원 임베딩을 만들고, 저차원 공간에서 또 다른 군집 알고리즘을 이용함

**9.2 가우시안 혼합**

* 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델
* 일반적으로 타원형
* 샘플마다 K개의 클러스터에서 랜덤하게 한 클러스터가 선택되고, j번째 클러스터에 할당된 샘플의 위치 x(i)는 평균이 M(j)이고 공분산 행렬이 sigma(j)인 가우시안 분포에서 랜덤 샘플링 됨

In [43]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [44]:
gm.weights_

array([0.21121839, 0.19597556, 0.59280605])

In [45]:
gm.means_

array([[ 1.71576174, -0.06897951],
       [-0.76442926,  0.54608058],
       [ 0.48215675,  0.26574216]])

In [46]:
gm.covariances_

array([[[ 0.05963712,  0.06608933],
        [ 0.06608933,  0.08980947]],

       [[ 0.0478498 ,  0.05827995],
        [ 0.05827995,  0.08523364]],

       [[ 0.16961252, -0.10302651],
        [-0.10302651,  0.2911442 ]]])

* 기댓값-최대화 알고리즘을 사용한 것
* 샘플을 클러스터에 랜덤하게 초기화, 클러스터를 계속하여 업데이트함
* 클러스터에 속할 추정 확률로 샘플에 가중치 적용됨

In [47]:
# 수렴 여부, 반복 횟수 확인
gm.converged_
gm.n_iter_

17

* 하드 군집(비슷한 클러스터에 할당), 소프트 군집(특정 클러스터에 속할 확률 예측)

In [48]:
# predict: 하드 군집, predict_proba: 소프트 군집
gm.predict(X)

array([0, 2, 2, 1, 1, 2, 2, 2, 2, 0, 1, 0, 0, 0, 0, 2, 0, 1, 2, 2, 0, 2,
       0, 2, 1, 0, 0, 0, 2, 2, 0, 1, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 1, 0, 1, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 1,
       0, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       1, 0, 2, 2, 2, 0, 2, 2, 0, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 1,
       2, 0, 1, 2, 2, 2, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 0, 1,
       2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 0, 0, 2, 2,
       2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 0, 2,
       2, 2, 1, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 0, 2, 1, 2, 2,
       2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 1, 1, 2, 1, 2, 0, 1, 2, 1, 2, 2,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0, 2, 0, 0, 1, 2, 0, 2, 1, 2,
       2, 0, 1, 2, 1, 2, 2, 2, 0, 2, 1, 2, 0, 2, 1, 0, 0, 2, 2, 1, 0, 2,
       1, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 2,

In [49]:
gm.predict_proba(X)

array([[9.59532043e-001, 3.83659992e-219, 4.04679571e-002],
       [1.89624740e-008, 7.08140233e-139, 9.99999981e-001],
       [3.59317118e-090, 2.74014486e-019, 1.00000000e+000],
       ...,
       [1.80149224e-008, 1.35396353e-138, 9.99999982e-001],
       [9.99664588e-001, 1.26144003e-222, 3.35411571e-004],
       [5.68606043e-106, 2.64601121e-012, 1.00000000e+000]])

* 가우시안 혼합 모델은 생성 모델로, 새로운 샘플을 만들어 낼 수 있음

In [50]:
X_new, y_new = gm.sample(6)
X_new

array([[ 1.61430182,  0.0194438 ],
       [-0.97600386,  0.25401993],
       [ 0.71455173, -0.45475097],
       [ 0.66375308, -0.66460207],
       [ 0.71589845,  0.94283521],
       [ 0.18577528,  0.94851213]])

In [51]:
y_new

array([0, 1, 2, 2, 2, 2])

* 주어진 위치에서 모델의 밀도 추정 가능 (그 위치의 확률 밀도 함수의 로그를 예측)

In [52]:
gm.score_samples(X)

array([-4.10216859e-01, -1.84271041e+00, -1.30803624e+00, -3.12931840e+00,
       -7.10206743e-01, -1.72315188e+00, -1.45218905e+00, -1.75461982e+00,
       -1.54946703e+00, -1.41596950e+00, -6.32448370e-01, -1.95343990e+00,
       -2.26990422e+00, -9.60813148e-01, -3.37932552e+00, -1.93856419e+00,
       -2.64357905e-01, -1.23603101e+00, -1.40629500e+00, -1.50197048e+00,
       -3.52291393e+00, -1.68064800e+00, -7.92117766e-01, -1.40331267e+00,
       -3.05639492e-01, -6.17238451e-01, -1.91870668e-01, -8.71168343e-01,
       -1.50250034e+00, -1.74420491e+00, -4.64941871e-01, -2.61409727e-01,
       -1.24450708e+00, -1.70755045e+00, -1.43713936e+00, -6.85728276e-01,
       -1.95891996e+00, -1.92949371e+00, -1.86802802e+00, -1.74467222e+00,
       -1.55887337e+00, -1.61188388e+00, -1.93644218e+00,  1.88745152e-02,
       -1.73996551e+00, -1.69557846e+00, -1.60950903e+00, -1.78802352e+00,
       -2.12175692e+00, -3.85008278e-01, -1.56933632e+00, -5.79904773e-01,
       -5.29369839e-01, -

* 클러스터의 모양과 방향의 범위를 제한 -> covariance_type에 spherical(원형), diag(타원형도 가능), tied(동일한 타원 모양, 크기, 방향)

**9.2.1 가우시안 혼합을 사용한 이상치 탐지**

* 밀도가 낮은 지역에 있는 모든 샘플을 이상치로 간주 -> 밀도 임곗값 필요
* 만약 거짓 양성이 너무 많으면 임곗값을 낮추고, 거짓 음성이 너무 많으면 임곗값을 더 높임
* 밀도 임곗값을 4%로 사용해 이상치 구분

In [53]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities,4)
anomalies = X[densities < density_threshold]

**9.2.2 클러스터 개수 선택하기**

* k-평균에 있는 이너셔나 실루엣 점수는 사용할 수 없음 -> bic, aic와 같은 이론적 정보 기준을 최소화하는 모델 찾기
* bic, aic는 모두 학습할 파라미터가 많은 모델에게 벌칙을 가하고 데이터에 잘 학습하는 모델에게 보상을 더함, 둘은 보통 비슷한 경향을 가지나, bic가 선택한 모델이 더 간단하고 그래서 더 잘 맞지 않을 수 있음

* AIC, BIC 계산

In [54]:
print(gm.bic(X), gm.aic(X))

2757.323669406574 2673.8918296638776


**9.2.3 베이즈 가우시안 혼합 모델**

* 최적의 클러스터 개수를 수동으로 찾지 않고 불필요한 클러스터의 가중치를 0으로 만드는 BayesianGaussianMixture 클래스를 사용할 수 있음
* 클러스터 개수를 최적의 클러스터 개수보다 크다고 믿을 만한 값으로 지정

In [3]:
import numpy as np
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_,2)

array([0.18, 0.13, 0.12, 0.15, 0.1 , 0.1 , 0.11, 0.12, 0.  , 0.  ])

->알고리즘이 자동으로 필요한 클러스터 개수를 인지함

* 위샤트 분포를 사용해 공분산 행렬 샘플링(d,V가 클러스터 분포 모양 제어)
* 잠재 변수 Z에 대한 사전 지식이 사전 확률이라는 확률 분포에 인코딩되고 클러스터가 적을 것이라는 사전 믿음을 가질 수 있음 -> 0.01 또는 10000으로 지정하면 매우 다른 군집 결과를 얻음(BUT 데이터가 많으면 그렇게 수치가 중요하지는 않음)

**9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘**

* PCA, Fast-MCD, 아이솔레이션 포레스트, LOF, one-class SVM